In [2]:
import pandas as pd
from datetime import datetime
import numpy as np
import time

In [3]:
# Latest Category
category_main = pd.read_excel("checkpoint_files/tracking_with_updated_category_daily_activity.xlsx")

# get only whatsapp and category
donor_category = category_main[['Whatsapp','Kategori']]

In [4]:
database_all = pd.read_excel("Database.xlsx")
database_all['Kategori'].value_counts()

Kategori
Aktif        54590
Pasif        12988
Invalid       1259
Eliminasi      552
Loyal          208
Name: count, dtype: int64

In [5]:
category_update = database_all[['Whatsapp','Kategori']].merge(donor_category,how='left', on='Whatsapp')
category_update

,Whatsapp,Kategori_x,Kategori_y
0,6285297062206,Aktif,AKTIF
1,62881082355438,Aktif,AKTIF
2,6281329198579,Pasif,NaN
3,6285725041295,Pasif,NaN
4,6285233090192,Pasif,NaN
...,...,...,...
66978,628240342524,NaN,NaN
66979,6285138033903,NaN,NaN
66980,6289682120531,NaN,NaN
66981,6281917672699,NaN,NaN


In [6]:
# def update_category(df):
#     # if Kategori_y isna take Kategori_x
#     if pd.isna(df['Kategori_y']):
#         return df['Kategori_x']
    
#     # if Kategori_x and Kategori_y are different take Kategori_y
#     elif df['Kategori_x'] != df['Kategori_y']:
#         return df['Kategori_y']
    
#     # if both are the same take Kategori_x
#     else:
#         return df['Kategori_x']

# category_update['Kategori'] = category_update.apply(update_category, axis=1)

# Start Here

In [5]:
category_update = pd.read_excel("checkpoint_files/category_each_donor.xlsx")

In [ ]:
# daily_1097 = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_1097.xlsx")
# daily_2056_bonding = pd.read_excel("../data_blast/daily_activity/filled/new/Daily Activity 2056 (BONDING).xlsx")
# daily_2056_maintain = pd.read_excel("../data_blast/daily_activity/filled/new/Daily Activity 2056 (MAINTAIN).xlsx")
# daily_2057_bonding = pd.read_excel("../data_blast/daily_activity/filled/new/Daily Activity 2057 (BONDING).xlsx")
# daily_2057_maintain = pd.read_excel("../data_blast/daily_activity/filled/new/Daily Activity 2057 (MAINTAIN).xlsx")
# daily_0046 = pd.read_excel("../data_blast/daily_activity/filled/new/0046_DA_Shania.xlsx")
# daily_0047 = pd.read_excel("../data_blast/daily_activity/filled/new/0047_DA_Shania.xlsx")
# daily_0041 = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_0041.xlsx")
# daily_0051 = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_0051.xlsx")
# daily_2031 = pd.read_excel("../data_blast/daily_activity/filled/old/Daily_Activity_WA_2031.xlsx")
# daily_2032 = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_wa_2032.xlsx")
# daily_7778 = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_7778.xlsx")

# November
# daily_0034 = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_0034.xlsx")
# daily_1097 = pd.read_excel("../data_blast/daily_activity/filled/new/Daily Activity Hanan 1097 DM DB.xlsx", sheet_name='Data Bonding')

# Desember
# daily_0046 = pd.read_excel("../data_blast/daily_activity/filled/new/0046_DA_Shania_update.xlsx", sheet_name="Donatur Bonding ")
# daily_0047 = pd.read_excel("../data_blast/daily_activity/filled/new/0047_DA_Shania_update.xlsx", sheet_name="Data Bonding")
# daily_2031 = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_WA_2031.xlsx", sheet_name="Donatur Baru Bonding ")
# daily_2032 = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_wa_2032.xlsx", sheet_name="Donatur Baru Bonding")
# daily_8196 = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_wa_8196.xlsx", sheet_name="Donatur Baru ")
# daily_0041_maintenance = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_0041.xlsx", sheet_name="Donatur Bonding")
# daily_1097_maintenance = pd.read_excel("../data_blast/daily_activity/filled/new/Daily Activity Hanan 1097 DM DB.xlsx", sheet_name="Data Maintenance")
# daily_1097_bonding = pd.read_excel("../data_blast/daily_activity/filled/new/Daily Activity Hanan 1097 DM DB.xlsx", sheet_name="Data Bonding")

# daily_2056 = pd.concat([daily_2056_bonding, daily_2056_maintain])
# daily_2057 = pd.concat([daily_2057_bonding, daily_2057_maintain])


daily_activity_all = pd.concat([daily_0041_maintenance, daily_1097_maintenance, daily_1097_bonding])
daily_activity_all = daily_activity_all.dropna(subset=['Whatsapp'])

c:\Users\faruq\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\faruq\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\faruq\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [8]:
daily_activity_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2154 entries, 0 to 769
Data columns (total 42 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   No                 2127 non-null   float64
 1   Tanggal            0 non-null      float64
 2   No CS              720 non-null    float64
 3   Donatur            2127 non-null   object 
 4   Whatsapp           2154 non-null   float64
 5   Donasi Sebelumnya  0 non-null      float64
 6   Periode Prospek    1490 non-null   object 
 7   Entri Donatur      1490 non-null   object 
 8   Tanggal 1          2127 non-null   object 
 9   Sesi Blast 1       637 non-null    object 
 10  Konten Blasting    2127 non-null   object 
 11  Action 1           2127 non-null   object 
 12  Respon 1           2125 non-null   object 
 13  Donasi 1           7 non-null      float64
 14  Tanggal 2          2089 non-null   object 
 15  Sesi Blast 2       599 non-null    object 
 16  Konten Blasting.1  1319 non-nu

In [9]:
update_category = category_update.merge(daily_activity_all[['Whatsapp','Next Action', 'Tanggal 3']], how='left', on='Whatsapp')

# standarize the value, for date and prioritas
update_category['Tanggal 3'] = pd.to_datetime(update_category['Tanggal 3'], errors='coerce')
update_category.loc[update_category['Next Action'] == "Prioirtas 3", "Next Action"] = "Prioritas 3"

# rename columns
# update_category = update_category.rename(columns={"Kategori":"kategori_update",
#                                                   "Next Action":"kategori_prioritas",
#                                                   "Tanggal 3":"terakhir_interaksi"})

In [10]:
update_category['Tanggal 3'].value_counts()

Tanggal 3
2025-11-28    555
2025-12-12    300
2025-11-21    298
2025-12-05    110
2025-12-04     70
2025-11-17     68
2025-11-20     67
2025-11-18     67
2025-11-19     66
2025-12-03     60
2026-11-26     53
2025-11-25     52
2025-11-24     50
2025-12-02     50
2025-11-29     50
2025-11-22     49
2025-11-26     37
2025-11-27     36
2025-12-13     23
Name: count, dtype: int64

In [12]:
# def category_update(df):
#     # if Kategori_y isna take Kategori_x
#     if pd.isna(df['Next Action']):
#         return df['kategori_prioritas']
    
#     # if Kategori_x and Kategori_y are different take Kategori_y
#     elif df['kategori_prioritas'] != df['Next Action']:
#         return df['Next Action']
    
#     # if both are the same take Kategori_x
#     else:
#         return df['kategori_prioritas']

update_category['kategori_prioritas'] = update_category['kategori_prioritas'].fillna(update_category['Next Action'])
update_category['terakhir_interaksi'] = update_category['terakhir_interaksi'].fillna(update_category['Tanggal 3'])

In [13]:
# update category when on the daily activity is eliminasi or invalid then return that value
update_category['kategori_update'] = np.where(update_category['kategori_prioritas'].isin(["Eliminasi", "Invalid"]), update_category['kategori_prioritas'], update_category['kategori_update'])

# update_category.loc[update_category['kategori_update'] == "AKTIF", "kategori_update"] = "Aktif"
# update_category.loc[update_category['kategori_update'] == "INVALID", "kategori_update"] = "Invalid"
# update_category.loc[update_category['kategori_update'] == "LOYAL", "kategori_update"] = "Loyal"
# update_category['kategori_update'].apply(lambda x: x if x in ["Eliminasi", "Invalid"] else )

In [14]:
update_category = update_category[['Whatsapp','kategori_update','kategori_prioritas','terakhir_interaksi']]

In [15]:
update_category = update_category.drop_duplicates(subset=['Whatsapp'])

In [18]:
update_category.to_excel("checkpoint_files/category_each_donor.xlsx", index=False)